<a href="https://colab.research.google.com/github/sadnow/RIFE-Google-Colab-Simplified/blob/main/motion_smoothing_VQGAN%2BCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplified RIFE Motion Smoothing (Trimmed for this speific use by sadnow)

For whatever reason this one makes the best motion smoothing with minimal messing around with setings.

Credits: Original script based on "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. Trimmed most of it out becuase I don't use the other parts.

Resources: Compression added from advice at https://unix.stackexchange.com/questions/28803/how-can-i-reduce-a-videos-size-with-ffmpeg as well as from https://stackoverflow.com/questions/61147016/concatenate-command-string-in-a-shell-script-in-google-colab 

This script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video.

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

# Mount Google Drive (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Motion interpolation (RIFE Smoothing)

"(...) (O)utput can be a bit wobbly, so video interpolation can be used to smooth things out a bit."


In [ ]:
print("Installing libraries for motion smoothing.")
!git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
!gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
!7z e RIFE_trained_model_HDv2.zip
!mkdir /content/RIFE/train_log
!mv *.pkl /content/RIFE/train_log/
%cd /content/RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt
%cd /content/
print("Done.")
%cd /content/RIFE/
print("Installing libraries for motion smoothing.")
# @title Settings
input_path='/content/drive/MyDrive/VQLIPSE/videos/original.mp4' #@param {type:"string"}
target_fps=60#@param {type:"raw"}
length_multiplier=2#@param {type:"integer"}
#@markdown
#@markdown `length_multiplier` might double, triple, or quadruple your video. Or square it? It's mysterious.
#@markdown
#@markdown ---
#@markdown Note: Your output will look like `original_2x_60fps.mp4` and can be found in your `input_path`.
#@markdown
#@markdown Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)

!python3 /content/RIFE/inference_video.py --fps=$target_fps --exp=$length_multiplier --video=$input_path --skip
%cd /content/

# x265 Conversion/Compression (Experimental, Very Fast)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss.

**Note: You manually need to input your desired compress_path (sorry) - will update this in the future w/ a checkbox to automatically determine your outputted interpolated file.**

In [ ]:
compress_path='/content/drive/MyDrive/VQLIPSE/videos/original_2x_60fps.mp4' #@param {type:"string"}
outputStr = '_x265-converted.mp4'
constant_quality=27#@param {type:"slider", min:20, max:50, step:1}

!pip install ffmpeg-python
#!ffmpeg -i $compress_path -vcodec hevc_nvenc -rc:v vbr_hq -cq $constant_quality $compress_path$outputStr -async 1 -vsync 1
!ffmpeg -i $compress_path -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $compress_path$outputStr 

#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality

#@markdown Note: Your output will look like `original_2x_60fps.mp4__x265-converted.mp4` and can be found in your `compress_path`.